In [6]:
## imports
import pandas as pd
#import tensorflow as tf
from os import *
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from skl import *

In [7]:
import numpy as np

In [130]:
# csv = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

# df = pd.DataFrame()

# wanted_cols = ['Offense Start DateTime', 'Offense Parent Group', 'Offense']

# for col in wanted_cols:
#     df[col] = csv[col]
#     df = df.dropna()

# df["Month"] = [pd.to_datetime(i).month for i in df['Offense Start DateTime']]

# df["Day"] = [pd.to_datetime(i).day for i in df['Offense Start DateTime']]

# df["Year"] = [pd.to_datetime(i).year for i in df['Offense Start DateTime']]

# df.drop(columns = ['Offense Start DateTime'], inplace = True)

# dfs = {}

df = pd.read_csv("df.csv")
df["Beat"] = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')['Beat']

def getBeat(df1, beat = ""):
    if beat == "":
        return df1
    return df1[df1["Beat"] == beat]

In [11]:
def compress(frame: pd.DataFrame, wanted_columns: list, category: str) -> pd.DataFrame:
    df = pd.DataFrame()
    df[category] = frame[category]
    for col in wanted_columns:
        df[col] = frame[col]
    return df.value_counts().to_frame()

In [9]:
## category is offense or offense parent category
## index for rows is years, for columns it is types
## value type is how many crimes
def one_param(category):
    l = df[[category, "Year"]]
    c2i = l.value_counts().to_frame()
    i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())
    tt = {}
    for i in i2c.iloc:
        if i[0] not in tt.keys():
            tt[i[0]] = {}
        else:
            if i[1] not in tt[i[0]].keys():
                tt[i[0]][i[1]] = 0
            tt[i[0]][i[1]] += int(c2i.loc[i[0]].loc[i[1]])
        
    return pd.DataFrame(tt).sort_index(ascending = False)

## multi-index dataframe
## main index is month, secondary index is year, columns are type
## value type is the percent
def two_param(category):
    l = df[[category, "Month", "Year"]]
    c2i = l.value_counts().to_frame()
    i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())
    tt = {}
    op = one_param(category)
    for i in i2c.iloc:
        if i[1] not in tt.keys():
            tt[i[1]] = {}
        else:
            if i[2] not in tt[i[1]]:
                tt[i[1]][i[2]] = {}
            else:
                if i[0] not in tt[i[1]][i[2]]:
                    tt[i[1]][i[2]][i[0]] = 0
                try:
                    tt[i[1]][i[2]][i[0]] += int(c2i.loc[i[0]].loc[i[1]].loc[i[2]]) / op[i[0]][i[2]]
                except:
                    pass
    tt = {k: pd.DataFrame(v) for k,v in tt.items()}
    tt = pd.concat(tt, axis=1)
    
    return tt.transpose().sort_index(ascending = True)

## returns a dictionary: keys are indexes for months, values
## are data frames:
## in each dataframe, multi-index, first index is day, second index
## is year, and value type is the percent
def three_param(category):
    ttt = {}
    l = df[[category, "Day", "Year", "Month"]]
    c2i = l.value_counts().to_frame()
    i2c = pd.DataFrame(l.value_counts().to_frame().index.to_list())
    tp = two_param(category)
    for m in range(1, 2):
        tt = {}
        n = l[l["Month"] == m].drop(columns = ["Month"])
        for i in i2c.iloc:
            if i[1] not in tt.keys():
                tt[i[1]] = {}
            else:
                if i[2] not in tt[i[1]]:
                    tt[i[1]][i[2]] = {}
                else:
                    if i[0] not in tt[i[1]][i[2]]:
                        tt[i[1]][i[2]][i[0]] = 0
                    try:
                        tt[i[1]][i[2]][i[0]] += float(c2i.loc[i[0]].loc[i[1]].loc[i[2]]) / tp[i[0]].iloc[i[3]].iloc[i[2]]
                    except:
                        pass
        tt = {k: pd.DataFrame(v) for k,v in tt.items()}
        tt = pd.concat(tt, axis=1)

        ttt[m] = tt.transpose().sort_index(ascending = True)
    return ttt

In [14]:
# three_param("Offense")

In [2]:
# opo = one_param("Offense")
# opoopg = one_param("Offense Parent Group")
# opoopg.to_csv("opoopg.csv")
# opo.to_csv("opo.csv")


NameError: name 'df' is not defined

In [ ]:
# tpo = two_param("Offense")
# tpoopg = two_param("Offense Parent Group")
# tpo.to_csv("tpo.csv")
# tpoopg.to_csv("tpoopg.csv")

In [ ]:
# thpo = three_param("Offense")
# thpoopg = three_param("Offense Parent Group")

1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
# for d in thpo:
#     thpo[d].to_csv(str(d)+"thpo.csv")
#     print(d)
# for d in thpoopg:
#     thpoopg[d].to_csv(str(d)+"thpoopg.csv")
#     print(d)

1
2
3
4
5
6
7
8
9
10
11
12
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
# df.to_csv("df.csv")

In [ ]:
# thpo[1].describe()

,Theft From Motor Vehicle,Burglary/Breaking & Entering,All Other Larceny,Motor Vehicle Theft,Destruction/Damage/Vandalism of Property,Simple Assault,Shoplifting,Theft of Motor Vehicle Parts or Accessories,Robbery,Drug/Narcotic Violations,...,Animal Cruelty,Incest,"Human Trafficking, Commercial Sex Acts",Drunkenness,Bribery,Betting/Wagering,Operating/Promoting/Assisting Gambling,"Human Trafficking, Involuntary Servitude",Negligent Manslaughter,Gambling Equipment Violation
count,516.0,529.0,543.0,510.0,520.0,516.0,504.0,513.0,505.0,495.0,...,81.0,41.0,33.0,19.0,16.0,20.0,5.0,2.0,17.0,1.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(3)

In [ ]:
l[l["Month"] == m].drop(columns = ["Month"])

,Offense,Year
97,Theft From Building,2019
243,All Other Larceny,2019
327,Theft of Motor Vehicle Parts or Accessories,2019
372,Credit Card/Automated Teller Machine Fraud,2019
396,Fondling,2019
...,...,...
1033211,Driving Under the Influence,2021
1033212,Stolen Property Offenses,2021
1033213,Weapon Law Violations,2021
1033286,Sexual Assault With An Object,2020


In [ ]:
one_param("Offense")

In [ ]:
two_param("Offense Parent Group")


In [ ]:
compress(df, ["Year", "Month", "Day"], "Offense")

In [ ]:
df.value_counts().to_frame()

In [ ]:
csv.columns

In [ ]:
df['Offense Start DateTime'].isna().value_counts()

False    1032212
Name: Offense Start DateTime, dtype: int64

In [ ]:
## datasets, data cleanup, test-train split
ds = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

## cleaning up the dates to have month, day, year columns
## also splitting it up into the layers of data
## which includes year, month, day

ds = ds[ds['Offense Start DateTime'] != None]

# dateTime = ds['Offense Start DateTime'].dropna()

ds["Month"] = [pd.to_datetime(i).month for i in ds['Offense Start DateTime']]

ds["Day"] = [pd.to_datetime(i).day for i in ds['Offense Start DateTime']]

ds["Year"] = [pd.to_datetime(i).year for i in ds['Offense Start DateTime']]

ds.loc[:, 'Crime Against Category':'Year']

In [ ]:
## train-test split
## to-do: for dsYear, Month, Day, create 3 splits
## each of 3 splits will be split 75% train and 25% test

In [ ]:
## polynomial regression
## to-do: learn how to do polynomial regression ,_,

In [ ]:
ds = pd.read_csv('../data/relevant/SPD_Crime_Data__2008-Present.csv')

In [ ]:
ds['Offense Start DateTime'].dropna()

In [ ]:
ds['Report DateTime'].isnull().value_counts()

False    1033415
Name: Report DateTime, dtype: int64

In [ ]:
ds["Year"] = [pd.to_datetime(i).year for i in ds['Offense Start DateTime']]

In [ ]:
opo = pd.read_csv("opo.csv")

In [ ]:
tpo = pd.read_csv("tpo.csv")

In [ ]:
tpo_dict = {}

In [ ]:
pd.read_csv("opo.csv")[["Unnamed: 0", "Burglary/Breaking & Entering"]]

In [10]:
pd.read_csv("9thpo.csv")

def a(df, df2, crime):
    df = pd.read_csv(df)
    n = df[["Unnamed: 0", "Unnamed: 1", crime]]
    dict1 = {}
    for i in range(1, 31):
        try:
            n = n[n["Unnamed: 0"] == i]
            m = n[["Unnamed: 1", crime]] / pd.read_csv(df2)[["Unnamed: 0", crime]]
            dict1[i] = m.describe().mean()[crime]
        except:
            pass
    return dict1

a("7thpo.csv", "opo.csv", "Burglary/Breaking & Entering")

{1: 0.625,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.0,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0}

In [ ]:
pd.read_csv("9thpo.csv")

,Unnamed: 0,Unnamed: 1,Theft From Motor Vehicle,Burglary/Breaking & Entering,All Other Larceny,Motor Vehicle Theft,Destruction/Damage/Vandalism of Property,Simple Assault,Shoplifting,Theft of Motor Vehicle Parts or Accessories,...,Animal Cruelty,Incest,"Human Trafficking, Commercial Sex Acts",Drunkenness,Bribery,Betting/Wagering,Operating/Promoting/Assisting Gambling,"Human Trafficking, Involuntary Servitude",Negligent Manslaughter,Gambling Equipment Violation
0,1,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,31,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,31,2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
740,31,2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,31,2022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
def a(df1, df2, crime):
    df1 = pd.read_csv(df1)
    n = df1[["Unnamed: 0", "Unnamed: 1", crime]]
    dict1 = {}
    for i in range(1, 31):
        try:
            n = n[n["Unnamed: 0"] == i]
            m = n[["Unnamed: 1", crime]] / pd.read_csv(df2)[["Unnamed: 0", crime]]
            dict1[i] = m.describe().loc['mean'][crime]
        except:
            pass
    return dict1

def average(df1, crime):
    if (df1.endswith("thpo.csv")):
        return a(df1, "opo.csv", crime)
    elif (df1.endswith("thpoopg.csv")):
        return a(df1, "opoopg.csv", crime)
    df1 = pd.read_csv(df1)
    n = df1[["Unnamed: 0", "Unnamed: 1", crime]]
    dict1 = {}
    for i in range(1, 13):
        dict1[i] = n[n["Unnamed: 0"] == i].describe().loc['mean'][crime]
    return dict1
    

In [ ]:
pd.read_csv("opo.csv")

,Unnamed: 0,Theft From Motor Vehicle,Burglary/Breaking & Entering,Identity Theft,Destruction/Damage/Vandalism of Property,Motor Vehicle Theft,All Other Larceny,Theft of Motor Vehicle Parts or Accessories,Simple Assault,Shoplifting,...,"Human Trafficking, Commercial Sex Acts",Drunkenness,Justifiable Homicide,Incest,Negligent Manslaughter,Betting/Wagering,Bribery,"Human Trafficking, Involuntary Servitude",Operating/Promoting/Assisting Gambling,Gambling Equipment Violation
0,2023,834.0,886.0,76.0,754.0,862.0,577.0,572.0,554.0,339.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,8927.0,8666.0,751.0,7236.0,NaN,NaN,NaN,5074.0,4323.0,...,12.0,NaN,4.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN
2,2021,9333.0,9823.0,823.0,NaN,5329.0,6239.0,5709.0,4683.0,3949.0,...,8.0,5.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
3,2020,9140.0,NaN,NaN,6658.0,4942.0,6102.0,3150.0,4755.0,3166.0,...,8.0,4.0,2.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN
4,2019,10757.0,7643.0,660.0,5804.0,3991.0,6667.0,1405.0,5593.0,4523.0,...,NaN,3.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN
5,2018,11714.0,8105.0,104.0,6651.0,4099.0,6560.0,1458.0,NaN,NaN,...,NaN,NaN,2.0,5.0,NaN,1.0,1.0,NaN,1.0,NaN
6,2017,12516.0,7790.0,48.0,6644.0,3705.0,5935.0,1393.0,5630.0,4767.0,...,2.0,NaN,3.0,2.0,NaN,3.0,2.0,NaN,NaN,NaN
7,2016,13191.0,7549.0,35.0,6322.0,4175.0,5569.0,1430.0,5475.0,4809.0,...,NaN,NaN,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,12683.0,7593.0,22.0,5983.0,3873.0,6144.0,1170.0,5278.0,4423.0,...,NaN,NaN,2.0,1.0,NaN,2.0,1.0,NaN,NaN,NaN
9,2014,NaN,7135.0,16.0,5750.0,5497.0,6358.0,1529.0,5060.0,4304.0,...,NaN,NaN,5.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN


In [ ]:
average("opo.csv", "Burglary/Breaking & Entering")

KeyError: "['Unnamed: 1'] not in index"

In [ ]:
average("8thpo.csv", "Burglary/Breaking & Entering")

{1: 2.375,
 2: 2.0,
 3: 2.125,
 4: 2.25,
 5: 2.125,
 6: 2.0,
 7: 2.125,
 8: 2.125,
 9: 2.0,
 10: 2.125,
 11: 2.375,
 12: 2.0}

In [ ]:
n = tpo[["Unnamed: 0", "Unnamed: 1", "Burglary/Breaking & Entering"]]

In [ ]:
n[n["Unnamed: 0"] == 1].describe().mean()["Burglary/Breaking & Entering"]

1.8478589907274814

In [ ]:
for i in tpo.iloc:
    if 

In [ ]:
tpo

,Unnamed: 0,Unnamed: 1,Burglary/Breaking & Entering,All Other Larceny,Destruction/Damage/Vandalism of Property,Motor Vehicle Theft,Simple Assault,Shoplifting,Aggravated Assault,Intimidation,...,Theft From Motor Vehicle,Hacking/Computer Invasion,Animal Cruelty,Purchasing Prostitution,Drunkenness,"Human Trafficking, Commercial Sex Acts",Bribery,Operating/Promoting/Assisting Gambling,Gambling Equipment Violation,"Human Trafficking, Involuntary Servitude"
0,1,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,12,2018,0.088957,0.098018,0.082845,0.081971,NaN,NaN,0.076477,0.073925,...,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
346,12,2019,0.085438,0.085346,0.078222,0.086194,0.079027,0.077161,0.087068,0.086123,...,NaN,0.140351,0.117647,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,12,2020,NaN,0.117666,0.080505,0.092473,0.068980,0.086229,0.073949,0.076469,...,0.082604,0.076923,0.115385,0.047619,NaN,NaN,NaN,NaN,NaN,NaN
348,12,2021,NaN,0.087193,NaN,0.100206,0.079863,0.092682,0.080145,NaN,...,0.090646,NaN,NaN,0.133333,NaN,0.125,NaN,NaN,NaN,NaN


In [12]:
## polynomial regression

# split the data 50/50

# for i in range 10
#     create the model 
#     calculate mse for 1/2 and 2/2
#     if abs(mse1-mse2) < curmin curmin = abs(mse1-mse2) and bestmodel = curmodel 

# settle on the last model

#------------LOOK AT THIS CODE!!!------------#
# loop through offenses

X = list(one_param("Offense").index)
y = list(one_param("Offense")[off].values())

X1, X2, y1, y2 = skl.train_test_split(X, y, test_size=0.5, train_size=0.5, random_state=None, shuffle=True, stratify=None)

# X1 = []  # for years 1/2
# y1 = []  # for occurences in that year 1/2

# X2 = []  # for years 2/2
# y2 = []  # for occurences in that year 2/2

bestmodel = None
curmin = int("inf")

for i in range(1, 11):
    curmodel = PolynomialRegression(degree = i)
    curmodel.fit(X1, y1)
    msetrain = mse(y1, curmodel.predict(X1))
    msetest = mse(y2, curmodel.predict(X2))

    diff = abs(msetrain-msetest)

    if diff < curmin:
        curmin = diff
        bestmodel = curmodel  

plt.scatter(X1, y1, color = 'r')
plt.plot(X2, y2, color = 'b')
plt.plot(X1, bestmodel.predict(X1), color = 'g')

#--------------------------------------------#

NameError: name 'off' is not defined

In [128]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

def predictCrime(crimeName, year, month, day, po):

    X = one_param("Offense").index
    y = one_param("Offense")[crimeName].values

    l = []
    for i in y:
        if str(i) == 'nan':
            l.append(0)
        else:
            l.append(i)
    
    y = l

    X1, X2, y1, y2 = train_test_split(X, y, test_size=0.5, train_size=0.5, random_state=None, shuffle=True, stratify=None)
    X1

    bestmodel = None
    curmin = float('inf')
    moe = 0
    fit = None
    features = 0

    for i in range(1, 11):
        #specify degree of 3 for polynomial regression model
        #include bias=False means don't force y-intercept to equal zero
        poly = PolynomialFeatures(degree=i, include_bias=False)

        #reshape data to work properly with sklearn
        poly_features = poly.fit_transform(np.array(X1).reshape(-1, 1))

        #fit polynomial regression model
        poly_reg_model = LinearRegression()
        poly_reg_model.fit(poly_features, y1)

        #display model coefficients
        print(poly_reg_model.intercept_, poly_reg_model.coef_)

        mse1 = mean_squared_error(y1, poly_reg_model.predict(poly_features))
        mse2 = mean_squared_error(y2, poly_reg_model.predict(poly.fit_transform(np.array(X2).reshape(-1, 1))))

        if abs(mse1-mse2) < curmin:
            curmin = abs(mse1-mse2)
            bestmodel = poly_reg_model
            moe = mse2
            fit = poly
            features = i
    n = average(str(month) + "th" + po + ".csv", crimeName)[day]
    if (str(n) == "nan") or n == 0.0:
        n = 0.6
    print(average("t"+po+".csv", crimeName)[month])
    print(n)
    poly2 = PolynomialFeatures(degree=features, include_bias=False)
    return bestmodel.predict(poly2.fit_transform(np.array(year).reshape(-1, 1))) * average("t"+po+".csv", crimeName)[month] * n, moe

res, moe = predictCrime('Robbery', 2025, 12, 3, "po")
print (res, moe)

-28010.60984933623 [14.33079135]
1927302.4605386388 [-1.97397549e+03  5.05284825e-01]
-65136184.222316846 [ 1.00186047e+05 -5.13584403e+01  8.77477266e-03]
-15945257.39670197 [ 1.91865879e-02  2.51480618e+01 -1.71876302e-02  3.30339626e-06]
-6243005.610563746 [ 4.35255672e-09  8.55757090e-06  8.41342470e-03 -6.46854140e-06
  1.32601752e-09]
-3054137.3604633193 [ 1.04082537e-15  2.37837806e-12  3.62154977e-09  3.16533431e-06
 -2.59569014e-09  5.54236972e-13]
-1706815.8412312153 [ 2.54722219e-22 -2.67864430e-18  1.18178694e-15  1.54939746e-12
  1.26973879e-09 -1.08454846e-12  2.38176339e-16]
-1042802.7817038691 [ 6.30954600e-29 -2.15976111e-20  3.48245347e-22  5.48303470e-19
  6.74019179e-16  5.30335126e-13 -4.65900982e-16  1.04441316e-19]
-679255.4212032955 [ 1.57379884e-35 -5.72807659e-24 -2.74066301e-28  1.70965189e-25
  2.52202161e-22  2.97664414e-19  2.27733099e-16 -2.04220903e-19
  4.65044277e-23]
-464329.8775306506 [ 3.94228405e-42 -1.48061501e-27  1.90457674e-31  4.99778820e-32
 